In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports

url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fnationalaccounts%2fbalanceofpayments%2fdatasets%2fuktradecountrybycommodityexports%2fcurrent/countrybycommodityexports.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "3. Monthly Exports", header=3, na_values = "X")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,2018JAN,2018FEB,2018MAR,2018APR,2018MAY,2018JUN,2018JUL,...,2021JUL,2021AUG,2021SEP,2021OCT,2021NOV,2021DEC,2022JAN,2022FEB,2022MAR,2022APR
0,0 Food & live animals,AD Andorra,EX Exports,0.00,0.00,0.01,0.00,0.00,0.00,0.01,...,0.01,0.02,0.04,0.01,0.00,0.01,0.00,0.02,0.02,0.00
1,0 Food & live animals,AE United Arab Emirates,EX Exports,17.30,15.40,19.16,16.63,16.46,14.40,17.79,...,20.54,20.90,17.61,18.30,20.70,21.50,20.20,15.95,22.36,23.31
2,0 Food & live animals,AF Afghanistan,EX Exports,0.00,0.00,0.04,0.00,0.15,0.00,0.16,...,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.09,0.00,0.16
3,0 Food & live animals,AG Antigua and Barbuda,EX Exports,0.19,0.27,0.25,0.19,0.14,0.33,0.21,...,0.23,0.19,0.29,0.14,0.25,0.68,0.66,0.71,0.89,0.48
4,0 Food & live animals,AI Anguilla,EX Exports,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "exports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.exports = df.exports*1000000

In [7]:
df.exports

0                  0.0
1           17300000.0
2                  0.0
3             190000.0
4                  0.0
              ...     
1540495      1000000.0
1540496      2000000.0
1540497    129000000.0
1540498      7000000.0
1540499      4000000.0
Name: exports, Length: 1540500, dtype: float64

In [8]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"exports": "sum"})

In [9]:
topcntry = test.sort_values(by = "exports", ascending = False).iloc[0:23].index.to_list()

In [10]:
df = df[df.COUNTRY.isin(topcntry)]

In [11]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [12]:
df

,COMMODITY,COUNTRY,time,exports
1,0 Food & live animals,United Arab Emirates,2018-01-01,1.730000e+07
15,0 Food & live animals,Total EU(28),2018-01-01,8.570000e+08
19,0 Food & live animals,Belgium,2018-01-01,4.396000e+07
36,0 Food & live animals,Canada,2018-01-01,1.390000e+07
41,0 Food & live animals,Switzerland,2018-01-01,5.800000e+06
...,...,...,...,...
1540447,Total,Sweden,2022-04-01,3.840000e+08
1540448,Total,Singapore,2022-04-01,5.180000e+08
1540474,Total,Turkey,2022-04-01,5.440000e+08
1540482,Total,United States,2022-04-01,4.233000e+09


In [13]:
grp = df.groupby(["COUNTRY"])

In [14]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["exports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["exports"]

        foo["exports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [15]:
df = grp.apply(ex_metal_total)

In [16]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [17]:
file = ".\\data"+ "\\UK-exports-1997-2017.parquet"

foo = pq.read_table(file).to_pandas()

In [18]:
df = df.append(foo) #add the historical data

In [19]:
df

,COUNTRY,COMMODITY,time,exports
0,Belgium,0 Food & live animals,2018-01-01,4.396000e+07
1,Belgium,00 Live animals,2018-01-01,1.000000e+05
2,Belgium,01 Meat & meat preparations,2018-01-01,5.910000e+06
3,Belgium,02 Dairy products & eggs,2018-01-01,7.770000e+06
4,Belgium,03 Fish & shellfish,2018-01-01,2.400000e+06
...,...,...,...,...
730291,Whole world,Total (ex. metals),2017-08-01,2.597800e+10
730292,Whole world,Total (ex. metals),2017-09-01,2.849800e+10
730293,Whole world,Total (ex. metals),2017-10-01,2.978700e+10
730294,Whole world,Total (ex. metals),2017-11-01,3.006700e+10


In [20]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [23]:
df.sort_index(inplace = True)

df.tail(50)

exports
COUNTRY     COMMODITY          time                    
Whole world Total (ex. metals) 2018-03-01  2.982500e+10
                               2018-04-01  2.756200e+10
                               2018-05-01  2.912600e+10
                               2018-06-01  2.975900e+10
                               2018-07-01  2.965100e+10
                               2018-08-01  2.800900e+10
                               2018-09-01  2.981600e+10
                               2018-10-01  3.210800e+10
                               2018-11-01  3.056800e+10
                               2018-12-01  2.669200e+10
                               2019-01-01  2.833800e+10
                               2019-02-01  2.880200e+10
                               2019-03-01  3.321200e+10
                               2019-04-01  2.670900e+10
                               2019-05-01  2.900300e+10
                               2019-06-01  2.778900e+10
                               2019-07-01  3.109000e+10
                               2019-08-01  2.846300e+10
                               2019-09-01  3.104800e+10
                               2019-10-01  3.405900e+10
                               2019-11-01  2.894400e+10
                               2019-12-01  2.858200e+10
                               2020-01-01  2.741700e+10
                               2020-02-01  2.654800e+10
                               2020-03-01  2.664800e+10
                               2020-04-01  1.912000e+10
                               2020-05-01  1.924500e+10
                               2020-06-01  2.297900e+10
                               2020-07-01  2.439200e+10
                               2020-08-01  2.249100e+10
                               2020-09-01  2.497900e+10
                               2020-10-01  2.797200e+10
                               2020-11-01  2.739700e+10
                               2020-12-01  2.742900e+10
                               2021-01-01  1.919300e+10
                               2021-02-01  2.361400e+10
                               2021-03-01  2.903200e+10
                               2021-04-01  2.588400e+10
                               2021-05-01  2.686600e+10
                               2021-06-01  2.671100e+10
                               2021-07-01  2.633200e+10
                               2021-08-01  2.324700e+10
                               2021-09-01  2.601600e+10
                               2021-10-01  2.893000e+10
                               2021-11-01  2.864400e+10
                               2021-12-01  2.886600e+10
                               2022-01-01  2.361300e+10
                               2022-02-01  2.810400e+10
                               2022-03-01  3.219500e+10
                               2022-04-01  3.025100e+10

In [25]:
df.index.get_level_values(0).unique()

Index(['Australia', 'Belgium', 'Canada', 'China',
       'Excl. EU 28 (Rest of World)', 'France', 'Germany', 'Hong Kong',
       'India', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'Norway', 'Poland',
       'Singapore', 'South Korea', 'Spain', 'Sweden', 'Switzerland',
       'Total EU(28)', 'Turkey', 'United Arab Emirates', 'United States',
       'Whole world'],
      dtype='object', name='COUNTRY')

In [26]:
out_file = ".\\data"+ "\\UK-exports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [27]:
file = ".\\data"+ "\\UK-exports.parquet"

foo = pq.read_table(file).to_pandas()

In [28]:
foo.head()

exports
COUNTRY   COMMODITY             time                 
Australia 0 Food & live animals 1997-01-01  4950000.0
                                1997-02-01  4870000.0
                                1997-03-01  3800000.0
                                1997-04-01  4640000.0
                                1997-05-01  4230000.0